<a href="https://colab.research.google.com/github/kishorex18/PytorchDL/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

In [17]:
class CNN(nn.Module):
  def __init__(self,in_channels=1,num_classes=10):
    super(CNN,self).__init__()
    self.conv1=nn.Conv2d(in_channels=1,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
    self.pool=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
    self.fc1=nn.Linear(16*7*7,num_classes)

  def forward(self,X):
    conv1_output=F.relu(self.conv1(X))
    pool1_output=self.pool(conv1_output)
    conv2_output=F.relu(self.conv2(pool1_output))
    pool2_output=self.pool(conv2_output)
    flattened=pool2_output.reshape(pool2_output.shape[0],-1)
    output=self.fc1(flattened)
    return output

In [5]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
input_size=28
learning_rate=0.001
epochs=2
batch_size=64
num_classes=10

In [25]:
train_data=MNIST(root="datasets/",train=True,transform=transforms.ToTensor(),download=True)
train_batches=DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_data=MNIST(root="datasets/",train=False,transform=transforms.ToTensor(),download=True)
test_batches=DataLoader(dataset=test_data,batch_size=batch_size,shuffle=True)

In [35]:
model=CNN().to(device)

In [36]:
creterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [37]:
for i in range(epochs):
  for batch_idx,(data,targets) in enumerate(train_batches):
    data=data.to(device)
    targets=targets.to(device)
    scores=model(data)
    loss=creterion(scores,targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

In [38]:
model.eval()

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

In [44]:
def check_accuracy(batches,model):
  num_samples=0
  num_correct=0
  model.eval()
  with torch.no_grad():
    for data,targets in batches:
      data=data.to(device)
      targets=targets.to(device)
      output=model(data)
      _,predicted=output.max(1)
      num_correct+=(targets==predicted).sum()
      num_samples+=predicted.size(0)

  model.train()
  return num_correct/num_samples

In [45]:
print(check_accuracy(test_batches,model))

tensor(0.9738)
